# Telco Company
---
Dataset: https://www.kaggle.com/datasets/blastchar/telco-customer-churn

Figure out what is the churn rate and customer lifetime value by product.

## Loading data and inital exploratory analysis

In [51]:
# Imports
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv('data/telco_churn.csv')

There are 3 types of contract:
- Month-to-month
- One year
- Two year

In [38]:
df.head(3)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,Churn_num
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No,0
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,No,No,No,One year,No,Mailed check,56.95,1889.5,No,0
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1


## Churn analysis

In [49]:
df['Churn_num'] = df['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)

In [52]:
# Churn ratio: Number of churned users / Number of total unique users
def my_func(x):
    return np.sum(x) / len(x)

In [56]:
df.groupby('Contract').agg({'customerID': 'count', 'MonthlyCharges': np.average, 'Churn_num': my_func})

,customerID,MonthlyCharges,Churn_num
Contract,,,
Month-to-month,3875,66.398490,0.427097
One year,1473,65.048608,0.112695
Two year,1695,60.770413,0.028319


Although the Monthly Charges average is similar to all 3 contract types, Month-to-month contracts have a much higher churn ratio (42,7%) when compared to the other 2 contracts (11,2% and 2,8%).

## Customer Lifetime Value

LTV = (Number of Months * Avg Order Value * Gross Margin * Avg Lifetime) / Number of Customers

In [58]:
gr = df.groupby('Contract').agg({'tenure': np.sum, 'customerID': 'count', 
                                 'MonthlyCharges': np.average, 'Churn_num': my_func})

In [60]:
gr['avg_lifetime'] = 1 / gr['Churn_num']

In [62]:
gr.columns = ['txns_count', 'customer_count', 'avg_month_charges', 'churn_pctg', 'avg_lifetime']

In [66]:
gross_margin = 0.60 # assumption. we estimate our profit will be 60% of our revenue.
gr['ltv'] = (gr['txns_count'] * gr['avg_month_charges'] * gross_margin * gr['avg_lifetime']) / gr['customer_count']
gr

,txns_count,customer_count,avg_month_charges,churn_pctg,avg_lifetime,ltv
Contract,,,,,,
Month-to-month,69892,3875,66.398490,0.427097,2.341390,1682.437445
One year,61932,1473,65.048608,0.112695,8.873494,14561.170150
Two year,96166,1695,60.770413,0.028319,35.312500,73050.594182


Our customers life time value by product would be:
- Monthly: 1.682 USD
- One year: 14.561 USD
- Two year: 73.050 USD